In [5]:
library(tidyverse)
library(tidymodels)

In [8]:
cleveland <- read_csv("https://raw.githubusercontent.com/Kooriryuu/dsci-100-project-group3/main/data/cleveland.data", col_names = FALSE, na = c("?"))
#switzerland <- read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.switzerland.data", col_names = FALSE, na = c("?"))
#cleveland <- read_csv("data/cleveland.data", col_names = FALSE, na = c("?"))
#switzerland <- read_csv("data/switzerland.data", col_names = FALSE, na = c("?"))

#cleveland
#switzerland

Rows: 303 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (14): X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X12, X13, X14

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 123 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (14): X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X12, X13, X14

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [23]:
cleveland_select <- cleveland |>
    select(X2, X3, X5, X4, X14)
#switzerland_select <- switzerland |>
    #select(X2, X3, X7, X11, X14)
#cleveland_select
#switzerland_select
column_names <- c("sex", "chest_pain_type", "serum_cholestoral", "resting_blood_pressure", "heart_disease_present")
colnames(cleveland_select) <- column_names
#colnames(switzerland_select) <- column_names
cleveland_select$heart_disease_present[cleveland_select$heart_disease_present > 1] <- 1

#cleveland_select <- mutate(cleveland_select, sex = as_factor(sex), chest_pain_type = as_factor(chest_pain_type), heart_disease_present = as_factor(heart_disease_present))
#cleveland_select




In [43]:
set.seed(77777) 
cleveland_split <- initial_split(cleveland_select, prop = 0.75, strata = heart_disease_present)
cleveland_training <- training(cleveland_split) 
cleveland_testing <- testing(cleveland_split) 

avg <-  cleveland_training |>
    select(serum_cholestoral, resting_blood_pressure) |>
    map_df(mean) 
count0 <- cleveland_training |>
    filter(heart_disease_present == 0) |>
    count() |>
    pull()
count1 <- cleveland_training |>
    filter(heart_disease_present == 1) |>
    count() |>
    pull()
avg <- mutate(avg, heart_disease_distr = count1 / (count0 + count1)) 
avg

serum_cholestoral,resting_blood_pressure,heart_disease_distr
<dbl>,<dbl>,<dbl>
244.8414,131.022,0.4581498
